In [1]:
from pathlib import Path

from pcd_dataset.kitti_dataset import KittiDataset

dataset_path = 'dataset/'
sequence = '00'
sam_labels_path = Path.cwd().joinpath('pipeline', 'vfm-labels', 'sam', sequence)

kitti = KittiDataset(dataset_path, sequence, sam_labels_path)

In [2]:
from mapping.simple_mapping import SimpleMapping

sm = SimpleMapping(kitti)
coo_matrix, pcds = sm.get_combined_labeled_point_clouds('cam2', 20, 23)

INFO:simple_mapping.SimpleMapping:Image 20 processed
INFO:simple_mapping.SimpleMapping:Image 21 processed
INFO:simple_mapping.SimpleMapping:Image 22 processed


In [3]:
from mapping.sparse_matrix_utils import construct_coo_matrix_for_multiple_views

coo_result = construct_coo_matrix_for_multiple_views(coo_matrix, 20)

In [4]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=True)

In [5]:
enc.fit((coo_result.T).toarray())

OneHotEncoder(handle_unknown='ignore')

In [6]:
enc.categories_

[array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
        27., 28., 29., 30., 31., 32., 33., 34., 35., 37., 38., 39., 40.,
        42., 43., 44., 45., 46., 48., 49., 50., 51., 53., 54., 56., 57.,
        58., 59., 62., 63., 64., 67., 68., 69., 71., 73., 74., 75., 76.,
        77., 78., 79., 80.]),
 array([ 0.,  2.,  3.,  4.,  5.,  6.,  7., 10., 11., 12., 13., 14., 15.,
        16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
        29., 30., 31., 32., 33., 34., 35., 36., 37., 39., 40., 41., 42.,
        43., 44., 45., 47., 49., 50., 51., 52., 54., 55., 56., 58., 59.,
        60., 62., 64., 65., 67., 69., 71., 72., 73., 74.]),
 array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
        26., 27., 28., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,
        40., 41., 42., 43., 44., 4

In [7]:
enc_coo_result = enc.transform(coo_result.T.toarray()).toarray()

In [8]:
inverse_enc_coo_result = enc.inverse_transform(enc_coo_result)

k = 0
n = 0
for itr in range(len(inverse_enc_coo_result)):
    if (inverse_enc_coo_result[itr] == [0.0, 0.0, 0.0]).all():
        k = k + 1
    if (inverse_enc_coo_result[itr] == [0.0, 0.0, 0.0]).any():
        n = n + 1
        
print("Number of points with non-zero all instances: {}".format(len(inverse_enc_coo_result) - k))
print("Number of points with non-zero any instances: {}".format(len(inverse_enc_coo_result) - n))

Number of points with non-zero all instances: 92411
Number of points with non-zero any instances: 1629


In [9]:
summary_pcd = pcds[0] + pcds[1] + pcds[2]

In [10]:
len(inverse_enc_coo_result)

832606

In [11]:
len(summary_pcd.points)

832606

In [12]:
inverse_enc_coo_result = enc.inverse_transform(enc_coo_result)
points = []
map_old_new_point_ind = {}

k = 0
for itr in range(len(inverse_enc_coo_result)):
    if (inverse_enc_coo_result[itr] == [0.0, 0.0, 0.0]).any():
        k = k + 1
    else:
        points.append(summary_pcd.points[itr])
        dict1 = {itr:(len(points) - 1)}
        map_old_new_point_ind.update(dict1)

In [13]:
len(map_old_new_point_ind)

1629

In [14]:
map_old_new_point_ind

{22: 0,
 23: 1,
 24: 2,
 25: 3,
 26: 4,
 27: 5,
 34: 6,
 35: 7,
 36: 8,
 37: 9,
 38: 10,
 39: 11,
 40: 12,
 41: 13,
 42: 14,
 43: 15,
 44: 16,
 45: 17,
 46: 18,
 47: 19,
 48: 20,
 49: 21,
 50: 22,
 51: 23,
 52: 24,
 53: 25,
 54: 26,
 55: 27,
 56: 28,
 57: 29,
 58: 30,
 59: 31,
 60: 32,
 61: 33,
 62: 34,
 63: 35,
 64: 36,
 65: 37,
 66: 38,
 67: 39,
 68: 40,
 69: 41,
 70: 42,
 71: 43,
 72: 44,
 73: 45,
 74: 46,
 75: 47,
 76: 48,
 77: 49,
 78: 50,
 79: 51,
 80: 52,
 81: 53,
 82: 54,
 83: 55,
 84: 56,
 85: 57,
 86: 58,
 87: 59,
 88: 60,
 89: 61,
 90: 62,
 91: 63,
 92: 64,
 93: 65,
 94: 66,
 95: 67,
 96: 68,
 97: 69,
 98: 70,
 99: 71,
 1248: 72,
 1249: 73,
 1250: 74,
 1251: 75,
 1252: 76,
 1253: 77,
 1254: 78,
 1255: 79,
 1256: 80,
 1257: 81,
 1258: 82,
 1259: 83,
 1260: 84,
 1261: 85,
 1262: 86,
 1263: 87,
 1264: 88,
 1265: 89,
 1266: 90,
 1267: 91,
 1268: 92,
 1269: 93,
 1270: 94,
 1271: 95,
 1272: 96,
 1273: 97,
 1274: 98,
 1275: 99,
 1276: 100,
 1277: 101,
 1278: 102,
 1279: 103,
 1280:

In [15]:
map_old_new_point_ind[235031]

988

In [16]:
enc_coo_non_zero_instances = []
for itr in range(len(enc_coo_result)):
    if itr in map_old_new_point_ind.keys():
        enc_coo_non_zero_instances.append(enc_coo_result[itr])

In [17]:
inverse_enc_coo_non_zero_instances = enc.inverse_transform(enc_coo_non_zero_instances)
print(inverse_enc_coo_non_zero_instances[988])
print(inverse_enc_coo_result[235031])

[12.  3.  2.]
[12.  3.  2.]


In [18]:
import open3d as o3d

pcd_baby = o3d.geometry.PointCloud()
pcd_baby.points = o3d.utility.Vector3dVector(points)

In [23]:
# downpcd = pcd_baby.voxel_down_sample(voxel_size=0.05)
# o3d.visualization.draw_geometries([downpcd])

In [34]:
# downpcd.points[50000]

array([-2.79080391,  1.66868822,  7.49760649])

In [33]:
# for it in range(len(downpcd.points)):
#     if (downpcd.points[it] == [-2.7908039129797504, 1.668688224242099, 7.497606486148587]).all():
#         print("ooo")

ooo


In [21]:
from sklearn.metrics.pairwise import pairwise_distances
distances = pairwise_distances(enc_coo_non_zero_instances)

In [22]:
distances.shape

(1629, 1629)

In [23]:
len(pcd_baby.points)

1629

In [27]:
pcd_baby.points[9]

array([-11.81667192,  -1.40100143,  27.68554811])

In [28]:
from awesome_normalized_cut import normalized_cut
import numpy as np

clusters = normalized_cut(distances, np.asarray(pcd_baby.points), 10)

1
(1629, 1629)
2
3
4
5
6
7
8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
012
11
1.0004602433313035
mask=[ True  True  True ...  True  True  True]
12
1
(1628, 1628)
2
3
4
5
6
7
8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
012
11
1.0002974471455892
mask=[False False False ...  True  True False]
12
1
(860, 860)
2
3
4
5
6
7
8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08

/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 3 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Users/sofiavivdich/projects/lidar-labelling/awesome_normalized_cut.py:17: RuntimeWarning: invalid value encountered in scalar divide
  return (cost / assoc_a) + (cost / assoc_b)
/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 4 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)


8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
012
11
inf
mask=[False False False]
15
13
1
(1, 1)
16
14
13
1
(12, 12)
2
3
4
5
6
7
8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
012
11
1.0115201109602867
mask=[ True False False False False  True  True False False  True False  True]
12
1
(5, 5)
2
3
4
5
6
7
8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004

/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 17 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 9 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)


7
8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
012
11
1.0471500655966388
mask=[ True False False  True False False  True False]
12
1
(3, 3)
2
3
4
5
6
7
8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
012
11
inf
mask=[False False False]
15
13
1
(5, 5)
2
3
4
5
6
7
8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
0

...

8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
012
11
1.0231374076204172
mask=[False  True  True False  True False False False False  True  True  True
  True  True  True  True  True  True  True False  True]
12
1
(14, 14)
2
3
4
5
6
7
8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
012
11
1.0975875900016274
mask=[ True  True  True  True  True  True  True  True  True  True  True False
 False False]
12
1
(11, 11)
2
3
4
5
6
7
8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
011

/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 6 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 10 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)


(4, 4)
2
3
4
5
6
7
8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
012
11
1.1645351568385462
mask=[ True False  True  True]
12
1
(3, 3)
2
3
4
5
6
7
8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
012
11
1.2085749884426302
mask=[ True False  True]
12
1
(2, 2)
16
13
1
(1, 1)
16
14
13
1
(1, 1)
16
14
13
1
(4, 4)
2
3
4
5
6
7
8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
0

/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 7 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)


004
010
012
11
1.0124859519980742
mask=[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False  True  True  True
  True  True]
12
1
(25, 25)
2
3
4
5
6
7
8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
012
11
1.014825820333311
mask=[False  True  True False  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True False  True  True  True  True
  True]
12
1
(21, 21)
2
3
4
5
6
7
8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001

In [29]:
clusters

[array([[-11.72434102,  -1.00536679,  17.02974917]]),
 array([[ 3.47427083, -0.39539425, 20.35702593],
        [ 2.16598842,  1.60863835, 11.58442374],
        [ 3.24206225,  1.12418292,  7.59515724]]),
 array([[6.00271842, 1.37492619, 7.15483902]]),
 array([[-11.81156238,  -1.42938674,  28.42363218],
        [  6.22916606,  -0.4327483 ,  24.64224851],
        [ -1.04503647,   1.60797694,   9.43636348],
        [ -1.07351997,   1.61660433,   9.18522634]]),
 array([[3.76772312, 0.99725983, 7.00850892]]),
 array([[ 6.46575694, -0.53599796,  9.01098904],
        [ 6.16200204, -0.45261387, 13.71487174],
        [-1.39873903,  1.61894656,  9.4086696 ]]),
 array([[ 6.22339422e+00, -5.77366853e-01,  7.78745028e+00],
        [-5.75596674e-03,  1.59041365e+00,  9.49129422e+00]]),
 array([[-11.76589232,  -1.34618364,  26.27454611],
        [  0.25301304,   1.58873503,  10.14403985]]),
 array([[-8.38348182, -0.07412376, 15.40347044],
        [ 6.29838922,  1.14205625,  7.43603542]]),
 array([[ 2.

In [55]:
inverse_enc_coo_non_zero_instances[61]

array([48., 41., 50.])

In [56]:
inverse_enc_coo_non_zero_instances[563]

array([ 0., 13., 30.])

In [43]:
clusters[0][9999]

array([9.29407103, 0.40789549, 0.18582554])

In [12]:
# 1) выбросить ненужные точки. создать новое облако и мапу старое - новое
# 2) voxel_down_sample применить к маленькому облаку
# применить к нему граф кат, раскрасить по кластерам ИСХОДНОЕ маленькое облако с новыми инстансами
# 3) kd деревья почитать чтобы найти соседей